<a href="https://colab.research.google.com/github/adithya70/-Fine-Tuning-AI-Models-with-LoRA-and-Deploying-with-Streamlit/blob/main/Fine_Tuning_AI_Models_with_LoRA_and_Deploying_with_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets peft torch torchaudio streamlit pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [17]:

!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/

In [18]:
!kaggle datasets download -d infamouscoder/depression-reddit-cleaned


Dataset URL: https://www.kaggle.com/datasets/infamouscoder/depression-reddit-cleaned
License(s): CC0-1.0
  0% 0.00/979k [00:00<?, ?B/s]
100% 979k/979k [00:00<00:00, 94.4MB/s]


In [19]:
!unzip depression-reddit-cleaned.zip -d data/


Archive:  depression-reddit-cleaned.zip
  inflating: data/depression_dataset_reddit_cleaned.csv  


In [20]:
import pandas as pd

# Load the dataset
df = pd.read_csv("/content/data/depression_dataset_reddit_cleaned.csv")
print(df.head())

                                          clean_text  is_depression
0  we understand that most people who reply immed...              1
1  welcome to r depression s check in post a plac...              1
2  anyone else instead of sleeping more when depr...              1
3  i ve kind of stuffed around a lot in my life d...              1
4  sleep is my greatest and most comforting escap...              1


In [22]:
df = df[['clean_text', 'is_depression']]

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

In [40]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Set pad token to eos token

# Model setup
model = AutoModelForCausalLM.from_pretrained("gpt2")
model.config.pad_token_id = tokenizer.pad_token_id  # Set pad token ID

# Prepare dataset
def prepare_dataset(examples):
    inputs = tokenizer(examples["clean_text"], truncation=True, padding="max_length", max_length=128)
    inputs["labels"] = inputs["input_ids"].copy()  # Labels should be the same as input_ids
    return inputs

dataset = dataset.map(prepare_dataset)

Map:   0%|          | 0/7731 [00:00<?, ? examples/s]

In [39]:
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id


In [33]:
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

# Load GPT-2 model
model = AutoModelForCausalLM.from_pretrained(model_name)

# Apply LoRA Configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["c_attn"],
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)

# Print trainable parameters
model.print_trainable_parameters()


trainable params: 294,912 || all params: 124,734,720 || trainable%: 0.2364


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [45]:
training_args = TrainingArguments(
    output_dir="./gpt2-lora-depression",
    per_device_train_batch_size=1,
    num_train_epochs=0.005,
    save_steps=5,
    logging_dir="./logs",
    logging_steps=5,
    report_to="none"  # Disabling reporting (useful if you don't have tracking)
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Start training
trainer.train()

Step,Training Loss
5,0.784900
10,1.078100
15,1.250700
20,0.190500
25,0.773300
30,1.465800
35,0.600800


TrainOutput(global_step=39, training_loss=0.8475653315201784, metrics={'train_runtime': 100.0574, 'train_samples_per_second': 0.386, 'train_steps_per_second': 0.39, 'total_flos': 2547597312000.0, 'train_loss': 0.8475653315201784, 'epoch': 0.005044625533566162})

In [46]:
# Save the model and tokenizer after training
model.save_pretrained("./gpt2-lora-depression-final")
tokenizer.save_pretrained("./gpt2-lora-depression-final")


('./gpt2-lora-depression-final/tokenizer_config.json',
 './gpt2-lora-depression-final/special_tokens_map.json',
 './gpt2-lora-depression-final/vocab.json',
 './gpt2-lora-depression-final/merges.txt',
 './gpt2-lora-depression-final/added_tokens.json',
 './gpt2-lora-depression-final/tokenizer.json')

In [48]:
%%writefile app.py
import streamlit as st
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the saved model and tokenizer
model = AutoModelForCausalLM.from_pretrained("./gpt2-lora-depression-final")
tokenizer = AutoTokenizer.from_pretrained("./gpt2-lora-depression-final")

# Set padding token to EOS token
tokenizer.pad_token = tokenizer.eos_token

# Function to generate text based on user input
def generate_text(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding="max_length", max_length=64)
    output = model.generate(inputs["input_ids"], max_length=max_length, num_return_sequences=1)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

# Streamlit app
st.title("GPT-2 LoRA Depression Model")

# Text input box for user prompt
user_input = st.text_area("Enter a prompt for text generation:", "")

# Button to trigger text generation
if st.button("Generate Text"):
    if user_input:
        generated_output = generate_text(user_input)
        st.write("Generated Text:")
        st.write(generated_output)
    else:
        st.write("Please enter a prompt to generate text.")


Writing app.py


In [52]:
!pip install pyngrok

In [53]:

from pyngrok import ngrok

# Set your ngrok auth token
ngrok.set_auth_token("2FmFzkDXLyfLTlWyu2zlfi8ovSw_2t74v3BtP2DUCd6uwD7Bk")

In [55]:
!streamlit run app.py & ngrok http 8501





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.191.194:8501

  Stopping...
Traceback (most recent call last):
  File "/usr/local/bin/ngrok", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pyngrok/ngrok.py", line 617, in main
    run(sys.argv[1:])
  File "/usr/local/lib/python3.11/dist-packages/pyngrok/ngrok.py", line 605, in run
    process.run_process(pyngrok_config.ngrok_path, args)
  File "/usr/local/lib/python3.11/dist-packages/pyngrok/process.py", line 306, in run_process
    subprocess.call(start)
  File "/usr/lib/python3.11/subprocess.py", line 391, in call
    return p.wait(timeout=timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/subprocess.py", line 1264, in wait
    return self._wait(timeout=timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/subprocess.py